
# MVP — *Machine Learning & Analytics*  
## Classificação de Sentimento em Textos de Cibersegurança (NLP)

**Aluno:** Marcus Rafael  
**Curso:** Ciência de Dados & Analytics – PUC-Rio  
**Data:** 2025-09-25
**Dataset:** [Hugging Face – zeroshot/cybersecurity-corpus](https://huggingface.co/datasets/zeroshot/cybersecurity-corpus)



# MVP — *Machine Learning & Analytics*  
## Classificação de Sentimento em Textos de Cibersegurança (NLP)

**Aluno:** Marcus Rafael  
**Curso:** Ciência de Dados & Analytics – PUC-Rio  
**Data:** 2025-09-25
**Dataset:** [Hugging Face – zeroshot/cybersecurity-corpus](https://huggingface.co/datasets/zeroshot/cybersecurity-corpus)



# MVP — *Machine Learning & Analytics*  
## Classificação de Sentimento em Textos de Cibersegurança (NLP)

**Aluno:** Marcus Rafael  
**Curso:** Ciência de Dados & Analytics – PUC-Rio  
**Data:** 2025-09-25
**Dataset:** [Hugging Face – zeroshot/cybersecurity-corpus](https://huggingface.co/datasets/zeroshot/cybersecurity-corpus)



## ✅ Checklist do MVP (o que precisa conter)
- Definição clara do problema e hipóteses.  
- Reprodutibilidade e ambiente configurados.  
- Carga e entendimento dos dados (EDA).  
- Definição do target, variáveis e divisão adequada dos dados.  
- Tratamento de dados e pipeline de pré-processamento.  
- Baseline e comparação com modelos candidatos.  
- Validação cruzada e otimização de hiperparâmetros.  
- Avaliação final, análise de erros e limitações.  
- Boas práticas e rastreabilidade.  
- Conclusões e próximos passos.  
- Artefatos salvos para reuso.



## 1. Escopo, objetivo e definição do problema

**Descrição:**  
Este projeto busca construir um modelo supervisionado para classificar sentimentos (0/1) em textos curtos sobre cibersegurança.

**Hipóteses:**  
- Classes podem estar desbalanceadas.  
- TF‑IDF com modelos lineares tende a ser eficaz.  
- F1‑macro é mais adequada que accuracy neste cenário.

**Condições:**  
- Dataset novo, não usado em aula.  
- Carregamento direto via URL pública.  
- Escopo: análise de NLP aplicada a segurança cibernética.


## 2. Reprodutibilidade e ambiente

In [1]:

import os, sys, platform, time, random, json, re
from pathlib import Path
import numpy as np, pandas as pd

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

import sklearn
from sklearn import set_config
set_config(transform_output="pandas")

import matplotlib.pyplot as plt

def env_report():
    return {
        "python": sys.version,
        "platform": platform.platform(),
        "numpy": np.__version__,
        "pandas": pd.__version__,
        "sklearn": sklearn.__version__,
    }

start_time = time.time()
env_report()


{'python': '3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]',
 'platform': 'Linux-6.6.97+-x86_64-with-glibc2.35',
 'numpy': '2.0.2',
 'pandas': '2.2.2',
 'sklearn': '1.6.1'}

## 3. Dados: carga, entendimento e qualidade

In [2]:

URL = "https://huggingface.co/datasets/zeroshot/cybersecurity-corpus/raw/main/dataset-unsplit.csv"
df = pd.read_csv(URL)
df.head()


,text,label
0,Microsoft leaks TLS private key for cloud ERP ...,0
1,U.S. Air Force Announces Third Bug Bounty Prog...,0
2,700K Guest Records Stolen in Choice Hotels Bre...,0
3,GoT Guide to Cybersecurity: Preparing for Batt...,0
4,"U.K. Teen Responsible for Bomb Threats, DDoS A...",0


### 3.1 Análise exploratória resumida (EDA)

In [3]:

print("Shape:", df.shape)
print(df.dtypes)
print("Nulos:", df.isna().sum().sum(), "Duplicados:", df.duplicated().sum())
print("Distribuição de classes:\n", df['label'].value_counts(normalize=True)*100)
df["text_len"] = df["text"].astype(str).str.len()
df["text_len"].describe()


Shape: (1000, 2)
text     object
label     int64
dtype: object
Nulos: 0 Duplicados: 0
Distribuição de classes:
 label
0    71.7
1    28.3
Name: proportion, dtype: float64


,text_len
count,1000.000000
mean,136.292000
std,66.511808
min,23.000000
25%,87.000000
50%,114.000000
75%,155.750000
max,304.000000


## 4. Definição do target, variáveis e divisão dos dados

In [4]:

from sklearn.model_selection import train_test_split, StratifiedKFold

X = df["text"].astype(str)
y = df["label"].astype(int)

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=SEED, stratify=y
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=SEED, stratify=y_temp
)

print(len(X_train), len(X_valid), len(X_test))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)


700 150 150


## 5. Tratamento de dados e Pipeline de pré-processamento

In [6]:

import re
from sklearn.feature_extraction.text import TfidfVectorizer

def clean_text(s: str) -> str:
    s = s.lower()
    s = re.sub(r"https?://\S+|www\.\S+", " ", s)
    s = re.sub(r"@[A-Za-z0-9_]+", " ", s)
    s = re.sub(r"#(\w+)", r"\1", s)
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_clean"] = df["text"].astype(str).apply(clean_text)
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9)


## 6. Baseline e modelos candidatos

In [7]:

from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import VotingClassifier

pipelines = {
    "dummy": Pipeline([("tfidf", tfidf), ("clf", DummyClassifier(strategy="most_frequent", random_state=SEED))]),
    "logreg": Pipeline([("tfidf", tfidf), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced", random_state=SEED))]),
    "svm_sgd": Pipeline([("tfidf", tfidf), ("clf", SGDClassifier(loss="hinge", max_iter=1000, random_state=SEED))]),
    "cnb": Pipeline([("tfidf", tfidf), ("clf", ComplementNB(alpha=0.5))]),
}
pipelines["voting"] = Pipeline([("tfidf", tfidf),
                                ("clf", VotingClassifier(estimators=[("lr", LogisticRegression(max_iter=1000)), ("cnb", ComplementNB())], voting="soft"))])


### 6.1 Treino e avaliação rápida (baseline vs candidatos)

In [8]:

from sklearn.model_selection import cross_validate
scoring = {"acc":"accuracy", "f1_macro":"f1_macro"}

cv_results = {}
for name, pipe in pipelines.items():
    scores = cross_validate(pipe, X_train, y_train, cv=cv, scoring=scoring, n_jobs=-1)
    cv_results[name] = {m: scores[f"test_{m}"].mean() for m in scoring}

pd.DataFrame(cv_results).T


,acc,f1_macro
dummy,0.717143,0.417635
logreg,0.772857,0.719485
svm_sgd,0.757143,0.689959
cnb,0.767143,0.711616
voting,0.784286,0.670302


## 7. Validação e Otimização de Hiperparâmetros

In [9]:

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, randint

search_space = {
    "tfidf__min_df": randint(1,5),
    "clf__C": loguniform(1e-2, 10)
}
search = RandomizedSearchCV(pipelines["logreg"], search_space, n_iter=10, cv=cv, scoring="f1_macro", random_state=SEED, n_jobs=-1)
search.fit(X_train, y_train)
best_model = search.best_estimator_
search.best_params_


{'clf__C': np.float64(1.3311216080736887), 'tfidf__min_df': 2}

## 8. Avaliação final, análise de erros e limitações

In [10]:

from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

best_model.fit(pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid]))
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-macro:", f1_score(y_test, y_pred, average="macro"))
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)


Accuracy: 0.8
F1-macro: 0.7482658312821661
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       108
           1       0.65      0.62      0.63        42

    accuracy                           0.80       150
   macro avg       0.75      0.74      0.75       150
weighted avg       0.80      0.80      0.80       150



array([[94, 14],
       [16, 26]])

## 11. Boas práticas e rastreabilidade

In [11]:

import joblib
ART_DIR = Path("artifacts"); ART_DIR.mkdir(exist_ok=True)

joblib.dump(best_model, ART_DIR / "best_model.joblib")
print("Modelo salvo em", ART_DIR)
print("Tempo total (min):", round((time.time()-start_time)/60,2))
env_report()


Modelo salvo em artifacts
Tempo total (min): 0.66


{'python': '3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]',
 'platform': 'Linux-6.6.97+-x86_64-with-glibc2.35',
 'numpy': '2.0.2',
 'pandas': '2.2.2',
 'sklearn': '1.6.1'}

## 12. Conclusões e próximos passos


- O modelo com melhor desempenho foi Logistic Regression otimizado, com F1‑macro superior ao baseline.  
- Textos curtos e ruidosos impactaram os resultados.  
- Próximos passos: testar modelos baseados em Transformers (BERT/DistilBERT), técnicas de data augmentation e maior limpeza textual.


## 13. Salvando artefatos (modelos e pipeline)

In [12]:

joblib.dump(best_model, ART_DIR / "pipeline_final.joblib")
with open(ART_DIR / "metrics.json", "w") as f:
    json.dump({"f1_macro": f1_score(y_test, y_pred, average="macro")}, f)
print("Artefatos salvos!")


Artefatos salvos!
